# Sentiment Analysis of Tweets by Brand <a class="anchor" id="Top"></a>
#### Authors: Eddie Prado, Sally Heinzel, Valeria Viscarra Fossati, and Weston Shuken


<img width="965" alt="Header Image" src="images/sentiment_analysis_header.png">

###### Image by SurveySensum

---

## Table of Contents
* [Overview](#Overview)
* [Business Opportunity](#Business_Opportunity)
* [Data & Methods](#Data_Methods)
* [Data Exploration](#Data_Exploration)
* [Data Modeling](#Data_Modeling)    
    * [Binary Predictor Modeling](#Binary)
        * [Baseline Models](#Binary_Baseline)
        * [Tuned Models](#Binary_Tuned)
    * [Multiclass Predictor Modeling](#Multiclass)
        * [Baseline Models](#Multiclass_Baseline)
        * [Tuned Models](#Multiclass_Tuned)
    * [Added Feature Modeling](#Added_features)
    * [Neural Networks](#Neural_Networks)
    * [Recurrent Neural Networks](#RNN)
* [Final Model](#Final_Model)
* [Results & Evaluation](#Results)
* [Online Application](#Online_App)
* [Recommendations](#Recommendations)
* [Next Steps](#Next_Steps)
* [Contact Us](#Contact)

## Overview <a class="anchor" id="Overview"></a>
Understanding brand and product reputation is difficult when only with provided customer surveys and review data. However, there is an abundance of social media responses to products and brands on various platforms. With these unofficial reviews and preferences towards products via tweets from Twitter, we can derive an overall sentiment towards your brand and products.

Our company, ViaGoGo, can provide you with a state-of-the-art machine learning model that rates the product and brand sentiment based on users who tweet about your brand. We can provide you with real-time graphs showing the trends of user sentiment towards your brand and products.

This notebook shows the entire process to creating and deploying our ML model for Twitter.



## Business Opportunity <a class="anchor" id="Business_Opportunity"></a>
Companies have little insight into their overall brand reputation on social media platforms. Twitter can provide real-time, accurate analysis of brand reputation based on the sentiment analysis of tweets on Twitter. The word cloud below is an example of how Twitter users are talking about Google and Apple:

<img width="965" alt="Wordcloud" src="https://user-images.githubusercontent.com/79488205/154748143-17743934-bc8a-40bc-8ec7-86d50fd60665.png">

Our team of experts has built a Machine Learning model that uses Natural Language Processing to distinguish between positive, neutral, and negative sentiment in Tweets. Using Google and Apple mentions on Twitter, we were able to classify sentiment to an 72% accuracy.

This model will be used as an analytics tool for companies to access their products' popularity on Twitter without having to access Twitter API. 


## Data & Methods <a class="anchor" id="Data_Methods"></a>
The dataset comes from Crowdflower via [data.world](https://data.world/crowdflower/brands-and-product-emotions) *Created: August 30, 2013 by Kent Cavender-Bares*. The data contains over 9,000 tweets from Twitter users that evaluated multiple brands and products. The crowd was asked if the tweet expressed positive, negative, or no emotion towards a brand and/or product. If some emotion was expressed, they were also asked to say which brand or product was the target of that emotion. 

During our exploratory data analysis, we found that the data was not balanced, as shown by the graphs below:

#### All data sentiment count:
<img width="410" alt="all_data_sentiment" src="https://user-images.githubusercontent.com/79488205/154748205-07024099-81f4-4a89-a86f-d5b8271ba599.png">

#### Google & Apple sentiment ratio:
<img width="411" alt="google_apple_sentiment_ratio" src="https://user-images.githubusercontent.com/79488205/154748394-608d0f0a-7ee0-465c-ab95-cbb2dccea9e3.png">

In order to address this imbalance, we first used a binary classifier to predict if a tweet would have a negative or not negative response. We chose to use this binary classification because negative sentiment is much more insightful to a brand versus neutral or positive sentiment. An example of usage could be to monitor if the negative sentiment increases on a particular day, then we can use inferential analysis to find specific tweets that were affecting the rating.

## Exploratory Data Analysis <a class="anchor" id="Data_Exploration"></a>

### Imports

In [ ]:
# Importing all needed libraries, packages, and modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from nlp_preprocessing import VIA_GoGo
import seaborn as sns
import gensim
import pickle
from textblob import TextBlob

from tensorflow.keras.layers import Dropout, Embedding, SpatialDropout1D, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Loading the dataset
tweet_df = pd.read_csv('data/db_tweet_emotion.csv', encoding= 'unicode_escape')

In [ ]:
# Viewing the dataset
tweet_df.head(5)

In [ ]:
# Seeing the datatypes and null values in the data
tweet_df.info()

In [ ]:
# Assigning variables of some of the columns for analysis
tweet = tweet_df['tweet_text']
directed = tweet_df['emotion_in_tweet_is_directed_at']
emotion = tweet_df['is_there_an_emotion_directed_at_a_brand_or_product']

In [ ]:
# Null values in the dataset per column
tweet_df.isna().sum()

In [ ]:
# Value counts of the emotion expressed in the tweet towards the brand or product
emotion.value_counts(normalize=True)

In [ ]:
# Plotting the value counts of the emotion towards the products and brands in the dataset
plt.barh(emotion.value_counts().index, emotion.value_counts().values);

In [ ]:
# Value counts of the tweet towards the brand or product
directed.value_counts(normalize=True)

In [ ]:
# Plotting the value counts of the products and brands in the dataset
plt.barh(directed.value_counts().index, directed.value_counts().values);

### Clean Up & Tokenize
In this section we will clean up the dataset, tweets, and tokenization.

In [ ]:
# Instantiating tokenizer
tokenizer = RegexpTokenizer(r"(?u)\b\w\w+\b")

# Create a list of stopwords in English
sw = stopwords.words('english')

In [ ]:
# Appending stop words to the stopwords list of words that appear frequently but don't hold much value
sw.append('link')
sw.append('rt')
sw.append('sxsw')
sw.append('quot')

In [ ]:
# instantiating Via_GoGo() -- see viagogo.py for this class
vg = VIA_GoGo()

# Creating a cleaned and shaped dataframe; This gives new columns 'tweet_text_tokenized', 'joined_tokens', 'emotion_num', & 'brand'. 
clean_df = vg.clean_tokenizer(tweet_df, tokenizer, sw)
clean_df # Viewing the clean dataset

In [ ]:
# Making columns that shows either negatvie (1) or not negative (0) sentiment 
clean_df['emotion_neg_not'] = clean_df['emotion_num']
clean_df['emotion_neg_not'] = clean_df['emotion_neg_not'].replace({0:1})
clean_df['emotion_neg_not'] = clean_df['emotion_neg_not'].replace({-1:0})

In [ ]:
# Viewing the imbalance in the binary column
clean_df.emotion_neg_not.value_counts(normalize=True)

In [ ]:
# Compared to the imbalance in the  multiclass dataset
clean_df.emotion_num.value_counts(normalize=True)

In [ ]:
# Creating new DataFrame that does not include neutral tweets
binary_df = clean_df[clean_df.emotion_num != 0]
# Viewing balance of classes when dropped neutral tweets
binary_df.emotion_num.value_counts(normalize=True)

In [ ]:
# Visualize the counts of each emotion type
sns.set_theme(style="darkgrid")
fig, ax = plt.subplots(figsize=(10,6))

ax = sns.countplot(
    data=binary_df, 
    x="directed_at",
    order=binary_df.brand.value_counts().index)

ax.set_title('Type of Emotions in Tweets',fontsize='x-large')
ax.set_xlabel('Emotion',fontsize='large')
ax.set_ylabel('Count',fontsize='large')
plt.xticks(range(2), ['Positive','Negative'],fontweight='light', fontsize='large')

plt.show();

In [ ]:
# Visualize the counts of each company discussed in tweets
sns.set_theme(style='darkgrid')
fig, ax = plt.subplots(figsize=(10,6))
ax = sns.countplot(
   data=clean_df,
   x='brand', order=clean_df['brand'].value_counts().index)
ax.set_title('Company Discussed in Tweet', fontsize='x-large')
ax.set_xlabel('Comapny/Product',fontsize='large')
ax.set_xticklabels(ax.get_xticklabels(),fontweight='light', fontsize='large')
ax.set_ylabel('Count',fontsize='large')
plt.show();


In [ ]:
# Visualize company/product mentioned by positive/non-positive emotion
import matplotlib.ticker as mtick
clean_df['pos_neg_neutral']=clean_df['emotion_num'] == 1
clean_df.groupby(['brand','pos_neg_neutral']).size().groupby(level=0).apply(
   lambda x: 100 * x / x.sum()).unstack().plot(kind='bar',stacked=True, figsize=(12,8), legend='reverse')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
plt.legend(bbox_to_anchor= (1.02, 1),prop = {'size' : 20},labels=('Not Positive', 'Positive'))
plt.xlabel('')
plt.xticks(fontsize=16,rotation=0,fontweight='light')
plt.yticks(fontsize=16)
plt.title('Percentage of Positive vs. Non-Positive Ratings by Company', size=25)
plt.show();

In [ ]:
#graph frequency of top 30 words
from sklearn.feature_extraction import text
#add custom words to stop word list
stop_words = text.ENGLISH_STOP_WORDS.union(['sxsw','mention','link','rt'])
cv = CountVectorizer(stop_words = stop_words)
words = cv.fit_transform(clean_df.joined_tokens)
sum_words = words.sum(axis=0)
words_freq = [(word, sum_words[0, i]) for word, i in cv.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
frequency = pd.DataFrame(words_freq, columns=['word', 'freq'])
frequency.head(30).plot(x='word', y='freq', kind='bar', figsize=(15, 7), color = 'blue')
plt.title('Most Frequently Occuring Words - Top 30', size=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16);

In [ ]:
# A word cloud of the tokenized words
from wordcloud import WordCloud
wordcloud = WordCloud(background_color = 'white', width = 1000, height = 1000).generate_from_frequencies(dict(words_freq))
plt.figure(figsize=(10,8))
plt.imshow(wordcloud)
plt.title('WordCloud - Vocabulary from Reviews', fontsize = 22);

In [ ]:
# Viewing top 15 words in only positive emotion tweets
frequency_dist_positive = FreqDist(clean_df["tweet_text_tokenized"][clean_df["emotion_num"] == 1].explode())

top_ten = list(zip(*frequency_dist_positive.most_common(15)))
tokens = top_ten[0]
counts = top_ten[1]

plt.figure(figsize=(15, 8))
plt.bar(tokens, counts)
plt.title('Top 15 Positive Tweet Word Occurences');

In [ ]:
# Viewing top 15 words in only neutral emotion tweets
frequency_dist_neutral = FreqDist(clean_df["tweet_text_tokenized"][clean_df["emotion_num"] == 0].explode())

top_ten = list(zip(*frequency_dist_neutral.most_common(15)))
tokens = top_ten[0]
counts = top_ten[1]

plt.figure(figsize=(15, 8))
plt.bar(tokens, counts)
plt.title('Top 15 Neutral Tweet Word Occurences');

In [ ]:
# Viewing top 15 words in only negative emotion tweets
frequency_dist_negative = FreqDist(clean_df["tweet_text_tokenized"][clean_df["emotion_num"] == -1].explode())

top_ten = list(zip(*frequency_dist_negative.most_common(15)))
tokens = top_ten[0]
counts = top_ten[1]

plt.figure(figsize=(15, 8))
plt.bar(tokens, counts)
plt.title('Top 15 Negative Tweet Word Occurences');

## Data Modeling <a class="anchor" id="Data_Modeling"></a>
First we will fit our binary prediction models on the negative and not negative predictors.
Then we will fit our muliclass prediction models on the positive, negative, and neutral predictors.

### Binary Data Predictor <a class="anchor" id="Binary"></a>

#### Baseline  <a class="anchor" id="Binary_Baseline"></a>

In [ ]:
# Baseline Linear Support Vector Machine
sgd = Pipeline([('vect', CountVectorizer()), 
               ('tfidf', TfidfTransformer()),
               ('clf', SGDClassifier()),
              ])

# Baseline Naive Bayes
naive = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

# Baseline Logistic Regression
lin_reg = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression()),
              ])
# Baseline Random Forest
rfc = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', RandomForestClassifier()),
              ])

# List of baseline models
baseline_models = [sgd, naive, lin_reg, rfc]


In [ ]:
# Creating empty metric table for attaching scores of model performence.
metric_table = pd.DataFrame(columns=['Model', 'CV Score', 'Test Accuracy', 'Type'])

In [ ]:
# Looping through baseline models and attaching scores to metric table
# This run_model() function is part of our ViaGoGo class in viagogo.py
for model in baseline_models:   
    row = vg.run_model(model, metric_table, clean_df['joined_tokens'], clean_df['emotion_neg_not'], 'binary_baseline', join_str=False)
    metric_table = pd.concat([row])

metric_table

#### Tuned  <a class="anchor" id="Binary_Tuned"></a>

In [ ]:
# Tuned Linear Support Vector Machine - w/ bigrams
sgd_tuned = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
               ('tfidf', TfidfTransformer(smooth_idf=False)),
               ('clf', SGDClassifier())
              ])
# Tuned Naive Bayes - w/ bigrams
naive_tuned = Pipeline([('vect', CountVectorizer(ngram_range=(2,2))),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
# Tuned Logistic Regression - w/ bigrams
lin_reg_tuned = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression(n_jobs=5)),
              ])
# Tuned Random Forest - w/ bigrams
rfc_tuned = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
               ('tfidf', TfidfTransformer()),
               ('clf', RandomForestClassifier(max_depth=800, n_estimators=100)),
              ])

# List of tuned models w/ bigrams
baseline_tuned_models = [sgd_tuned, naive_tuned, lin_reg_tuned, rfc_tuned]

In [ ]:
# Looping through tuned models and attaching scores to metric table
for model in baseline_tuned_models:   
    row = vg.run_model(model, metric_table, clean_df['joined_tokens'], clean_df['emotion_neg_not'], 'binary_tuned', join_str=False)
    metric_table = pd.concat([row])

metric_table

In [ ]:
# Plotting all the binary models
vg.plot_models(metric_table)

#### Binary Modeling Takeaways:
- The accuracy did not improve much over the imbalance in the dataset of 93.7% not negative values.
- We will proceed with adding the neutral values back in for multiclass predictions.

### Multiclass Data Predictor  <a class="anchor" id="Multiclass"></a>

#### Baseline  <a class="anchor" id="Multiclasss_Baseline"></a>

In [ ]:
# Baseline Linear Support Vector Machine 
sgd_mc = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
               ('tfidf', TfidfTransformer(smooth_idf=False)),
               ('clf', SGDClassifier())
              ])
# Baseline Naive Bayes
naive_mc = Pipeline([('vect', CountVectorizer(ngram_range=(2,2))),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
# Baseline Logistic Regression
lin_reg_mc = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression(solver='newton-cg', n_jobs=5)),
              ])
# Baseline Random Forest
rfc_mc = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
               ('tfidf', TfidfTransformer()),
               ('clf', RandomForestClassifier(max_depth=800, n_estimators=100, class_weight='balanced')),
              ])

# List of multiclass baseline models
multiclass_models = [sgd_mc, naive_mc, lin_reg_mc, rfc_mc]

In [ ]:
# New metric table for multiclass models
metric_table_multi = pd.DataFrame(columns=['Model', 'CV Score', 'Test Accuracy', 'Type'])

In [ ]:
# Looping through baseline models and attaching scores to metric table
for model in multiclass_models:   
    row = vg.run_model(model, metric_table_multi, clean_df['joined_tokens'], clean_df['emotion_num'], 'baseline_multiclass', join_str=False)
    metric_table_multi = pd.concat([row])

metric_table_multi

#### Tuned  <a class="anchor" id="Multiclass_Tuned"></a>

In [ ]:
# Baseline Linear Support Vector Machine
sgd_tuned_mc = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
               ('tfidf', TfidfTransformer(smooth_idf=False)),
               ('clf', SGDClassifier())
              ])
# Baseline Naive Bayes
naive_tuned_mc = Pipeline([('vect', CountVectorizer(ngram_range=(2,2))),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
# Baseline Logistic Regression
lin_reg_tuned_mc = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression(solver='newton-cg', n_jobs=5)),
              ])
# Baseline Random Forest
rfc_tuned_mc = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
               ('tfidf', TfidfTransformer()),
               ('clf', RandomForestClassifier(max_depth=800, n_estimators=100, class_weight='balanced')),
              ])

# List of multiclass tuned models w/ bigrams
multiclass_models_tuned = [sgd_tuned_mc, naive_tuned_mc, lin_reg_tuned_mc, rfc_tuned_mc]

In [ ]:
# Looping through tuned models and attaching scores to metric table
for model in multiclass_models_tuned:   
    row = vg.run_model(model, metric_table_multi, clean_df['joined_tokens'], clean_df['emotion_num'], 'tuned_multiclass', join_str=False)
    metric_table_multi = pd.concat([row])

metric_table_multi

In [ ]:
# Plotting all the multiclass models
vg.plot_models(metric_table_multi)

#### Multiclass Modeling Takeaways:
- The accuracy is lower than the binary models which is expected given the imbalance in the dataset. 
- Our model is performing over 10% better than always predicting the most prevelant class.
- Results may vary slightly because we did not specify random state in each classifier.
- We will try some more strategies for modeling below.

### Adding Features <a class="anchor" id="Added_Features"></a>
In this section, we add some features to out dataset to see if it will improve performence.

In [ ]:
# Using TextBlob for getting polarity score
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

clean_df['TextBlob_Polarity'] = clean_df['joined_tokens'].apply(getPolarity)

In [ ]:
# Using Afinn to get sentiment score
from afinn import Afinn
afinn = Afinn(language='en')
def getAffinscore(text):
    return afinn.score(text)
clean_df['Affin_score'] = clean_df['joined_tokens'].apply(getAffinscore)

In [ ]:
# Viewing these features
sentiment_features = clean_df.loc[:,['TextBlob_Polarity', 'Affin_score']]
sentiment_features

In [ ]:
# Joining these features to the dataframs and vectorizing for modeling
clean_df['joined_tokens'] = clean_df['joined_tokens'].str.replace('\d+', '')

X = clean_df['joined_tokens']
countvec = CountVectorizer(min_df=5, ngram_range=(1,2), stop_words='english', strip_accents='unicode')


X_vec = countvec.fit_transform(X)

X_df = pd.DataFrame(X_vec.toarray(), columns=countvec.get_feature_names())
preprocess_features = sentiment_features.join(X_df, on=X_df.index)

In [ ]:
# Modeling for new features
sgd_feat = SGDClassifier()

rfc_feat = RandomForestClassifier(max_depth=100, n_estimators=100, class_weight='balanced')

feature_models = [sgd_feat, rfc_feat]

In [ ]:
# Looping through new features models and attaching scores to metric table
for model in feature_models:   
    row = vg.run_model(model, metric_table_multi, preprocess_features, clean_df['emotion_num'], 'features_multiclass', join_str=False, pipeline=False)
    metric_table_multi = pd.concat([row])

metric_table_multi

### Word2Vec <a class="anchor" id="Word2Vec"></a>

In [ ]:
# Using Word2Vec Strategy for improving score -- code shoutout https://github.com/prateekjoshi565/twitter_sentiment_analysis/blob/master/code_sentiment_analysis.ipynb
model_w2v = gensim.models.Word2Vec(
            sentences=clean_df.tweet_text_tokenized,
            vector_size=200, # desired no. of features/independent variables 
            window=5, # context window size
            min_count=2,
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 2, # no.of cores
            seed = 34)

model_w2v.train(clean_df.tweet_text_tokenized, total_examples=len(clean_df.tweet_text_tokenized), epochs=20)

In [ ]:
# Seeing similarity score to different terms
model_w2v.wv.most_similar(positive="iphone")

In [ ]:
model_w2v.wv.most_similar(positive="awesome")

In [ ]:
# Creating function for creating vector of the tokens
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v.wv[word].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary
                         
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:
# Putting it all together into a word2vec dataframe
wordvec_arrays = np.zeros((len(clean_df.tweet_text_tokenized), 200))

for i in range(len(clean_df.emotion_num)):
    wordvec_arrays[i,:] = word_vector(clean_df.tweet_text_tokenized[i], 200)
    
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape  # printing shape 

In [ ]:
# Modeling for Word2Vec
sgd_word2vec = SGDClassifier()
rfc_word2vec = RandomForestClassifier(class_weight='balanced')

word2vec_models = [sgd_word2vec, rfc_word2vec]

In [ ]:
# Looping through Word2Vec models and attaching scores to metric table
for model in word2vec_models:   
    row = vg.run_model(model, metric_table_multi, wordvec_df, clean_df['emotion_num'], 'word2vec', join_str=False, pipeline=False)
    metric_table_multi = pd.concat([row])
    
metric_table_multi


### Neural Network  <a class="anchor" id="Neural_Network"></a>

In [ ]:
# Creating new column of targets for NN
clean_df['emotion_num_nn'] = clean_df['emotion_num']

In [ ]:
# Need to replace the -1 with 2 for negative sentiment
clean_df['emotion_num_nn'] = clean_df['emotion_num_nn'].replace(-1, 2)

In [ ]:
# Isolating X and y variables
X = clean_df['joined_tokens']
y =  clean_df['emotion_num_nn']

In [ ]:
# Splitting to test and train sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    test_size= 0.25,
                                                    random_state= 5)

In [ ]:
# Vectorizing and transforming with TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1,2))

X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

# View the shapes of X variables
print('Shape of train features tensor:', X_train_vec.shape)
print('Shape of test features tensor:', X_test_vec.shape)

# View the shapes of y variables
print('Shape of train label tensor:', y_train.shape)
print('Shape of test label tensor:', y_test.shape)

In [ ]:
# Creating Neural Network Model
model = Sequential()
model.add(layers.Dense(300, input_dim=(38060), activation='relu' ))
model.add(Dropout(.4))
model.add(layers.Dense(3, activation='softmax'))

In [ ]:
# Compiling and running Neural Network
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train_vec, y_train, epochs=6, batch_size=200, validation_split=.20, verbose=1)

In [ ]:
# Printing test score accuracy
results = model.evaluate(X_test_vec, y_test, batch_size=10)
print("test loss, test acc:", results)

In [ ]:
# Plotting loss of train and validation sets by epochs
plt.figure(figsize=(8,5))
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.show();

In [ ]:
# Plotting accuracy of train and validatio sets by epochs
plt.figure(figsize=(8,5))
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show();

In [ ]:
# Appending results to metric table
NN_dict = {'Model': 'NeuralNetwork', 
            'CV Score': max(history.history['val_accuracy']),
            'Test Accuracy': results[1],
            'Type': 'multiclass_NN'}
NN_row = pd.DataFrame(NN_dict, index=[0])

metric_table_multi = pd.concat([metric_table_multi, NN_row], ignore_index=True)

### Recurrent Neural Network w/ LSTM <a class="anchor" id="RNN"></a>

In [ ]:
# Creating variables for out Recurrent Neural Network -- Code shoutout to Susan Li https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

# Instantiating Tokenizer and fiting to 'joined_tokens'
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(clean_df['joined_tokens'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
# Tokenizing strings and padding sequeneces to the same length -- "MAX_SEQUENCE_LENGTH"
X = tokenizer.texts_to_sequences(clean_df['joined_tokens'].values) # Transforms each text in texts to a sequence of integers
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape) # Printing shape of X

In [ ]:
# Creating dummy variables for sentiment values of 'emotion_num'
y = pd.get_dummies(clean_df['emotion_num']).values
print('Shape of label tensor:', y.shape) # Printing Y shape

In [ ]:
# Splitting to test and training sets -- printing shapes of each variable
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=.2, random_state=5)
print(X_train.shape,Y_train.shape) 
print(X_test.shape,Y_test.shape)

In [ ]:
# Instantiating Sequential Model 
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(4))
model.add(Dense(3, activation='softmax'))

# Compiling and running Recurrent Neural Network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Defining epochs and batch_size
epochs = 6
batch_size = 200

# Fitting the model
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,
                    validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', 
                                                                  patience=3, min_delta=0.0001)])

In [ ]:
# Plotting loss of train and validation sets by epochs
plt.figure(figsize=(8,5))
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
# Plotting accuracy of train and validatio sets by epochs
plt.figure(figsize=(8,5))
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
# Appending results to metric table
RNN_dict = {'Model': 'RecurrentNeuralNetwork', 
            'CV Score': max(history.history['val_accuracy']),
            'Test Accuracy': 0,
            'Type': 'multiclass_NN'}
RNN_row = pd.DataFrame(RNN_dict, index=[0])

metric_table_multi = pd.concat([metric_table_multi, RNN_row], ignore_index=True)

In [ ]:
# Viewing all the models
metric_table_multi

In [ ]:
# Plotting all the multiclass models
vg.plot_models(metric_table_multi)

#### Modeling Strategies Takeaways:
- With all of the work done, we still have great results from out SGDClassifier.
- We will go with this model for the final model, since the accuracy score is high, but also the computing power and time to run is quicker than most other models.

### Final Model <a class="anchor" id="Final_Model"></a>

For our final model, we chose to use the top performing on the multiclass in order to predict any of the three classes of tweet sentiment.

In [ ]:
# Finding our top MULTICLASS MODEL
metric_table_multi.sort_values(by=['Test Accuracy'], ascending=False)

In [ ]:
# Splitting new variables into train and test sets to evaluate our final model.
X_train, X_test, y_train, y_test = train_test_split(clean_df['joined_tokens'], 
                                                    clean_df['emotion_num'], 
                                                    test_size= 0.25,
                                                    random_state= 5)

In [ ]:
# Creating variables for the final predictions and final true values
final_preds = sgd_tuned_mc.predict(X_test)
final_trues = y_test

In [ ]:
# Plotting the confusion matrix for the final model
fig, ax = plt.subplots(figsize=(8, 8))
plot_confusion_matrix(sgd_tuned_mc, X_test, y_test, ax=ax, display_labels=['neg', 'neutral', 'pos'])
plt.title('Final Model Confusion Matrix')
plt.grid(False)
plt.show();

In [ ]:
# Saving the model for usage in our application -- app.py
pickle.dump(sgd_tuned_mc, open('final_clf.pkl', 'wb'))

## Results & Evaluation <a class="anchor" id="Results"></a>

We built multiclass models to predict whether tweets were positive, neutral and negative. Our baseline model has a 61%  accuracy which is derived by always predicting the neutral class (given its imbalance). We increased this score using machine learning models, specifically Naive Bayes, Random Forest, Neural Network, and SGD Classifier, finally arriving at our best model at ~72% accuracy.

<img width="405" alt="multiclass_models" src="https://user-images.githubusercontent.com/79488205/154748935-457f5a4e-bf1e-4199-9a80-f0d99913e04d.png">


## Online Application

We created an online application to demo the model's performance. The user can input a sample tweet and click the 'GoGo!' button, which will then return a sentiment score for the sample tweet of Positive, Negative, or Neutral.

<img width="1200" alt="App Photo" src="images/app_photo.png">


To demo the product, please click on our logo or visit this [link]().

## Recommendations <a class="anchor" id="Recommendations"></a>
Being competitive in the 21st century means utilizing 21st-century tools. ViaGogo’s Twitter Sentiment Analysis, built using natural language processing, offers Twitter an opportunity to give their brand users an advantage in the marketplace. This product enables businesses to capture public reactions about their company and products in a far more timely and authentic manner than focus groups or surveys. It collects and analyzes real-time reactions in order for businesses to make effective decisions.

The Twitter Sentiment Analysis adds value to businesses in 3 main ways:
-	Brand Perception – track what people are saying about a company/product in real-time within a mercurial social media environment;
-	Market Research – identify and explore the sentiments directed at one’s competitors in order to develop strategies based on their successes and struggles;
-	Customer Service – pinpoint which brands, locations, or services are thriving in customer satisfaction and which ones need the most urgent attention.

By adopting ViaGogo’s Twitter Sentiment Analysis and offering it as a service for corporate users, Twitter will increase its utility and make itself an indispensable part of the modern business landscape.


## Next Steps <a class="anchor" id="Next_Steps"></a>
ViaGogo in currently working on some new applications for its Twitter Sentiment Analysis. One product in development is the Positivity Rater. This tool gives users a positivity rating based on the analysis of the account’s past tweets. This can be useful to increase engagement among Twitters users. With so many people turned off by negative and harmful content, the Positivity Rater allows an individual to gauge how positive someone is when considering whether or not to follow them. We believe this will boost user activity and retain those who otherwise may stop using Twitter due to so much unwanted, pernicious content.

Another area ViaGoGo is actively working on is the creation of dashboards, which allow users to easily visualize sentiment analysis both in real-time and longitudinally. These dashboards will filter data on different dimensions, such as time frame, location, and product/service. It will also facilitate easy comparison of a given metric (for example, comparing the sentiments expressed towards 2 different branches or how sentiment about a product compares to this time last year).

Finally, ViaGoGo is already in production of a web app that allows users to write a tweet and see its sentiment rating before publishing it. This will ensure that the author strikes the appropriate tone with their tweet and can prevent thoughtless or poorly constructed tweets from entering the public domain. This feature will be valued by social media managers sitewide.

---
<h2><center>Brought to you by</center></h2>
<center><img width="175" alt="Header Image" align="center" src="images/ViaGoGo_logo.png" ></center>

---


#### For more information <a class="anchor" id="Contact"></a>
Please contact the contributors to this analysis: 
[Eddie Prado]() |
[Sally Heinzel]() |
[Valeria Viscarra Fossati](https://www.linkedin.com/in/valeria-vf/) |
[Weston Shuken](https://www.linkedin.com/in/westonshuken/)

[Return to top](#Top)